# Scraping Article Information from HTML Website 

#### Install needed packages

In [12]:
conda install -c anaconda requests

Solving environment: done

## Package Plan ##

  environment location: /Applications/anaconda2/envs/py3

  added / updated specs:
    - requests


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    brotlipy-0.7.0             |py37haf1e3a3_1000         357 KB  anaconda
    ca-certificates-2020.6.24  |                0         132 KB  anaconda
    certifi-2020.6.20          |           py37_0         159 KB  anaconda
    cffi-1.14.0                |   py37hb5b8e2f_0         219 KB  anaconda
    chardet-3.0.4              |        py37_1003         173 KB  anaconda
    cryptography-2.9.2         |   py37ha12b0ac_0         608 KB  anaconda
    idna-2.10                  |             py_0          56 KB  anaconda
    openssl-1.1.1g             |       h1de35cc_0         3.4 MB  anaconda
    pycparser-2.20             |             py_2          94 KB  anaconda
    pyopenssl-19.1.0           |  

In [16]:
conda install -c anaconda bs4 -y

Solving environment: done

## Package Plan ##

  environment location: /Applications/anaconda2/envs/py3

  added / updated specs:
    - bs4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.9.1       |           py37_0         167 KB  anaconda
    bs4-4.9.1                  |                0           4 KB  anaconda
    soupsieve-2.0.1            |             py_0          33 KB  anaconda
    ------------------------------------------------------------
                                           Total:         204 KB

The following NEW packages will be INSTALLED:

  beautifulsoup4     anaconda/osx-64::beautifulsoup4-4.9.1-py37_0
  bs4                anaconda/noarch::bs4-4.9.1-0
  soupsieve          anaconda/noarch::soupsieve-2.0.1-py_0



soupsieve-2.0.1      | 33 KB     | ##################################### | 100% 
beautifulsoup4-4.9.1 | 167 KB    | #################

In [138]:
conda install -c anaconda xlsxwriter

Solving environment: done

## Package Plan ##

  environment location: /Applications/anaconda2/envs/py3

  added / updated specs:
    - xlsxwriter


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    xlsxwriter-1.2.9           |             py_0         108 KB  anaconda
    ------------------------------------------------------------
                                           Total:         108 KB

The following NEW packages will be INSTALLED:

  xlsxwriter         anaconda/noarch::xlsxwriter-1.2.9-py_0



xlsxwriter-1.2.9     | 108 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done

Note: you may need to restart the kernel to use updated packages.


### Import needed package

In [139]:
import csv
import requests 
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import xlsxwriter

### Input the required vol and issue of the articles
In out case, we extract articles information from Vol 51 Issue 7 to Vol 52 Issue 6

In [15]:
vol = [52,51] 
vol = np.repeat(vol,6)
issue = list(range(1,13))
vol_no = pd.DataFrame(vol,columns = {'vol'})
vol_no['issue'] = issue
vol_no #check the corresponding vol_issue 

,vol,issue
0,52,1
1,52,2
2,52,3
3,52,4
4,52,5
5,52,6
6,51,7
7,51,8
8,51,9
9,51,10


### Extract Article Information

In [143]:
IISE_total_list_July2019toJune2020 = pd.DataFrame()

for i in range(0,len(vol_no)):
    url0 = ['https://www.tandfonline.com/toc/uiie21/'+vol_no['vol'][i].astype(str)+'/'+vol_no['issue'][i].astype(str)+'?nav=tocList']
    url = url0[0]
    
    response = requests.get(url) #Getting the response from mentioned URL using get() method of requests 
    html = response.content
    soup = BeautifulSoup(html) 
    
    title = [item.get_text() for item in soup.select("span.hlFld-Title")]
    Author = [item.get_text() for item in soup.select("span.articleEntryAuthorsLinks")]
    doi = [a['href'] for a in soup.select("a.ref.nowrap")]
    doi = pd.DataFrame(doi)
    doi.columns = doi.columns.get_level_values(0)
    #doi = doi[doi[0].str.contains('pdf')] #creat the access to the pdf directly
    doi = doi[doi[0].str.contains('full')] #access to the full article webpage
    doi.drop_duplicates(inplace = True)
    Link = ['https://www.tandfonline.com'+doi]
    
    iise_paper_vol_no = pd.DataFrame()
    iise_paper_vol_no['Volume'] = np.repeat(vol_no['vol'][i],len(title))
    iise_paper_vol_no['Issue'] = np.repeat(vol_no['issue'][i],len(title))
    iise_paper_vol_no['Article'] = title
    iise_paper_vol_no['Authors'] = Author
    link = Link[0]
    link.reset_index(drop=True, inplace=True)
    iise_paper_vol_no = pd.concat([iise_paper_vol_no,link],axis=1,ignore_index=True)  
    
    IISE_total_list_July2019toJune2020 = IISE_total_list_July2019toJune2020.append(iise_paper_vol_no)

Rename the columns

In [146]:
IISE_total_list_July2019toJune2020.rename(columns={0: "Volume", 1: "Issue",2:"Article",3:"Authors",4:"Link"},inplace = True)

### Save the Article Information to .xlsx File

In [147]:
IISE_total_list_July2019toJune2020.to_excel("./IISE_total_list_July2019toJune2020.xlsx") 